In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import copy
import os
import time
from os import path as osp

import numpy as np
import sacred
import torch
import yaml
from sacred import Experiment
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from tracktor.config import get_output_dir
from tracktor.datasets.factory import Datasets
from tracktor.datasets.evaluation_wrapper import evaluationSequence
from tracktor.frcnn_fpn import FRCNN_FPN
from tracktor.oracle_tracker import OracleTracker
from tracktor.reid.resnet import ReIDNetwork_resnet50
from tracktor.tracker import Tracker
from tracktor.utils import (evaluate_mot_accums, get_mot_accum,
                            interpolate_tracks, plot_sequence)

In [3]:
import cv2

In [4]:
frame_split = [0.0, 1.0]

In [5]:
obj_detects = []
obj_detect_model = "../../output/custom/model_epoch_30.model"
obj_detect = FRCNN_FPN(num_classes=31)
obj_detect.load_state_dict(torch.load(obj_detect_model, map_location=lambda storage, loc: storage))
obj_detects.append(obj_detect)
obj_detect.eval()
obj_detect.cuda();

In [6]:
tracker_conf = {
  "detection_person_thresh": 0.4,
  "regression_person_thresh": 0.05,
  "detection_nms_thresh": 0.1,
  "regression_nms_thresh": 0.,
  "motion_model" : {
    "enabled": True,
    "n_steps": 5,
    "center_only": True
  },
  "public_detections": False,
  "do_align": True,
  "warp_mode": "MOTION_EUCLIDEAN",
  "number_of_iterations": 100,
  "termination_eps": 0.00001,
  "do_reid": False,
  "inactive_patience": 50,
  "max_features_num": 10,
  "reid_sim_threshold": 2.0,
  "reid_iou_threshold": 0.2
}
tracker = Tracker(obj_detect, [], tracker_conf)
tracker.reset()
num_frames = 0

In [7]:
from google.cloud import storage
import csv
import pandas as pd
from io import StringIO, BytesIO

PROJECT_NAME = "pluto-a31d9"
BUCKET_NAME = "pluto-tracking-samples"
client = storage.Client(project=PROJECT_NAME)
bucket = client.get_bucket(BUCKET_NAME)

groundtruth = bucket.get_blob("3_roads_faxe_gt_annotations.csv")
model_output = bucket.get_blob("3_roads_faxe_model_output.csv")

annotations_df = pd.read_csv(StringIO(groundtruth.download_as_string().decode("utf-8")))

# Image id is in model output file:
model_output_df = pd.read_csv(StringIO(model_output.download_as_string().decode("utf-8")))

annotations_df = annotations_df.merge(
    model_output_df[['image', 'image_id']].drop_duplicates(['image_id']),
    on='image_id'
)

# "CaptureId" column is expected, we can use the`image_id` as "proxy":
annotations_df["CaptureId"] = annotations_df['image_id']

In [8]:
name = "strandvejen_faxe"
result_dir = "../../output/custom/results"
seq = evaluationSequence(name, result_dir, annotations_df)
data_loader = DataLoader(seq, batch_size=1, shuffle=False)

{1: '0001_38c09e0b-acc5-492f-8b1d-b2a58826ca37_img', 15: '0015_22562779-770b-4345-a7b9-47665532e8cb_img', 29: '0029_3a1e940e-2fcf-4e8e-ac74-dfdf2e77df58_img', 43: '0043_d416fc0e-b543-4331-9f52-0791e5212255_img', 57: '0057_137542c9-4e92-4393-93d3-371a21a1f5a4_img', 70: '0070_f8011f5d-eea9-4cbf-bb77-7261c267bde6_img', 83: '0083_9d35bc85-4be0-4c3e-87dd-5daca7bfe8b5_img', 94: '0094_1d570f56-b421-4890-822f-b9540813a112_img', 105: '0105_90c7456a-2c6d-4886-b460-32167ac6b4f1_img', 115: '0115_893a3846-10fa-4622-aa63-40c711c8f93c_img', 125: '0125_e0bf32c7-7d2c-4a0a-b9d6-2d4f61c0141d_img', 135: '0135_d0f2f88d-c110-4339-b212-a915c346e235_img', 145: '0145_ab8f9845-b16a-427a-a9d2-10ebe905497a_img', 155: '0155_9c7f2dfb-5824-4b6e-ab8c-f5ae146db4fa_img', 165: '0165_f5134cd3-01d6-4023-b417-94fceb58bc1d_img', 175: '0175_b34a1ec3-e0e6-45dd-8359-350da023c686_img', 186: '0186_570eaa2c-51c0-4cc8-a617-3f80bcfac3f4_img', 196: '0196_7db3d5cc-0f22-44aa-afad-8b0b0edb9cf0_img', 206: '0206_9ac8593c-c951-4061-aa45-a

In [9]:
annotations_df

,image_id,ClassName,Confidence,X1,X2,Y1,Y2,AnnotationId,label,image,CaptureId
0,e4e81487-8128-40c4-a6e2-bd693e4eeba4,13,0.40625,1,146,653,795,7106da40-9e87-4d08-bfdc-f93104f040ab,R30,0001_e4e81487-8128-40c4-a6e2-bd693e4eeba4_img,e4e81487-8128-40c4-a6e2-bd693e4eeba4
1,e45d6a0b-5e68-43f7-9df0-eb6d302bb1a8,13,1.00000,2,246,666,862,c2a95d03-fb5d-44a0-a68f-228aabe35456,R30,0040_e45d6a0b-5e68-43f7-9df0-eb6d302bb1a8_img,e45d6a0b-5e68-43f7-9df0-eb6d302bb1a8
2,732f21b0-c7b8-4165-aecc-bce391eb4c7f,13,1.00000,2,285,680,890,669a03bd-92f9-46ec-ab91-2967488c50fe,R30,0066_732f21b0-c7b8-4165-aecc-bce391eb4c7f_img,732f21b0-c7b8-4165-aecc-bce391eb4c7f
3,c9a141e6-9830-4f62-b39b-cc298aea4ecd,17,1.00000,110,177,714,733,ceefc4ac-bb7c-4bf8-9842-0de8ff97dacc,M10,0094_c9a141e6-9830-4f62-b39b-cc298aea4ecd_img,c9a141e6-9830-4f62-b39b-cc298aea4ecd
4,c9a141e6-9830-4f62-b39b-cc298aea4ecd,13,0.50000,1,279,723,928,a59fa0e5-2129-4d05-bf18-fa11b1231373,R30,0094_c9a141e6-9830-4f62-b39b-cc298aea4ecd_img,c9a141e6-9830-4f62-b39b-cc298aea4ecd
...,...,...,...,...,...,...,...,...,...,...,...
1988,3a560861-ea6c-42e6-acbe-1176b7209419,9,1.00000,920,1024,825,922,43a53de5-dbbe-40ff-afca-4b7354b5a2f8,D20,1542_3a560861-ea6c-42e6-acbe-1176b7209419_img,3a560861-ea6c-42e6-acbe-1176b7209419
1989,3a560861-ea6c-42e6-acbe-1176b7209419,9,1.00000,725,886,821,881,f57bdea3-4dc2-4417-9c35-4dadc35410c7,D20,1542_3a560861-ea6c-42e6-acbe-1176b7209419_img,3a560861-ea6c-42e6-acbe-1176b7209419
1990,3a560861-ea6c-42e6-acbe-1176b7209419,17,1.00000,103,165,710,724,83708299-c522-4d74-a91c-a6296547adbc,M10,1542_3a560861-ea6c-42e6-acbe-1176b7209419_img,3a560861-ea6c-42e6-acbe-1176b7209419
1991,3a560861-ea6c-42e6-acbe-1176b7209419,9,1.00000,0,377,683,1019,e35e5787-b9c3-47b8-aadb-1f2b5e2b347e,D20,1542_3a560861-ea6c-42e6-acbe-1176b7209419_img,3a560861-ea6c-42e6-acbe-1176b7209419


In [10]:
results = seq.load_results()
count = 0
for i, frame in enumerate(tqdm(data_loader, leave=True, position=0)):
    count += 1
    if len(seq) * frame_split[0] <= i <= len(seq) * frame_split[1]:
        with torch.no_grad():
            tracker.step(frame, -1)
        num_frames += 1
        if num_frames == 100:
            break
    if count > 200: pass

results = tracker.get_results()
results = interpolate_tracks(results)
seq.write_results(results)

zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj
zdxfcghvj


In [11]:
results[0]

{0: array([414.52920532, 882.6295166 , 686.15368652, 926.09344482,
         16.        ,   0.97681707])}

In [17]:
for row in annotations_df.values:
    print(row)

['e4e81487-8128-40c4-a6e2-bd693e4eeba4' 13 0.40625 1 146 653 795
 '7106da40-9e87-4d08-bfdc-f93104f040ab' 'R30'
 '0001_e4e81487-8128-40c4-a6e2-bd693e4eeba4_img'
 'e4e81487-8128-40c4-a6e2-bd693e4eeba4']
['e45d6a0b-5e68-43f7-9df0-eb6d302bb1a8' 13 1.0 2 246 666 862
 'c2a95d03-fb5d-44a0-a68f-228aabe35456' 'R30'
 '0040_e45d6a0b-5e68-43f7-9df0-eb6d302bb1a8_img'
 'e45d6a0b-5e68-43f7-9df0-eb6d302bb1a8']
['732f21b0-c7b8-4165-aecc-bce391eb4c7f' 13 1.0 2 285 680 890
 '669a03bd-92f9-46ec-ab91-2967488c50fe' 'R30'
 '0066_732f21b0-c7b8-4165-aecc-bce391eb4c7f_img'
 '732f21b0-c7b8-4165-aecc-bce391eb4c7f']
['c9a141e6-9830-4f62-b39b-cc298aea4ecd' 17 1.0 110 177 714 733
 'ceefc4ac-bb7c-4bf8-9842-0de8ff97dacc' 'M10'
 '0094_c9a141e6-9830-4f62-b39b-cc298aea4ecd_img'
 'c9a141e6-9830-4f62-b39b-cc298aea4ecd']
['c9a141e6-9830-4f62-b39b-cc298aea4ecd' 13 0.5 1 279 723 928
 'a59fa0e5-2129-4d05-bf18-fa11b1231373' 'R30'
 '0094_c9a141e6-9830-4f62-b39b-cc298aea4ecd_img'
 'c9a141e6-9830-4f62-b39b-cc298aea4ecd']
['b8ee687

 'b34a1ec3-e0e6-45dd-8359-350da023c686']
['b34a1ec3-e0e6-45dd-8359-350da023c686' 10 0.5 538 569 868 965
 '956f2e12-18b6-4052-9005-2c26fcbc83f3' 'D10'
 '0175_b34a1ec3-e0e6-45dd-8359-350da023c686_img'
 'b34a1ec3-e0e6-45dd-8359-350da023c686']
['570eaa2c-51c0-4cc8-a617-3f80bcfac3f4' 16 1.0 554 764 911 1023
 '6f307c99-d0f2-43a0-b577-931d0de89b51' 'R10'
 '0186_570eaa2c-51c0-4cc8-a617-3f80bcfac3f4_img'
 '570eaa2c-51c0-4cc8-a617-3f80bcfac3f4']
['570eaa2c-51c0-4cc8-a617-3f80bcfac3f4' 29 1.0 562 633 922 963
 '8cecf9b7-6158-4dc1-b120-92eee9406b9d' 'R20'
 '0186_570eaa2c-51c0-4cc8-a617-3f80bcfac3f4_img'
 '570eaa2c-51c0-4cc8-a617-3f80bcfac3f4']
['570eaa2c-51c0-4cc8-a617-3f80bcfac3f4' 10 1.0 660 695 910 940
 '476dc9d1-a671-4c44-a706-dd7eaf35ef71' 'D10'
 '0186_570eaa2c-51c0-4cc8-a617-3f80bcfac3f4_img'
 '570eaa2c-51c0-4cc8-a617-3f80bcfac3f4']
['7db3d5cc-0f22-44aa-afad-8b0b0edb9cf0' 10 1.0 703 736 963 1015
 '7b82d2db-109f-4ec3-a7e7-e06bee71d7f9' 'D10'
 '0196_7db3d5cc-0f22-44aa-afad-8b0b0edb9cf0_img'
 '7